In [1]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from skimage.transform import resize
import os

In [2]:
def lpq(img_filepath, winSize=3,freqestim=1,mode='im'):

    img = cv2.imread(img_filepath,cv2.IMREAD_GRAYSCALE)
    img =  (resize(img, (110, 200))*255).astype(np.uint8)
    rho=0.90

    STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
    sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
    sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

    convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

    img=np.float64(img) # Convert np.image to double
    r=(winSize-1)/2 # Get radius from window size
    x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

    if freqestim==1:  #  STFT uniform window
        #  Basic STFT filters
        w0=np.ones_like(x)
        w1=np.exp(-2*np.pi*x*STFTalpha*1j)
        w2=np.conj(w1)

    ## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
    # Run first filters
    filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
    filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
    filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
    filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

    # Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
    freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                        filterResp2.real, filterResp2.imag,
                        filterResp3.real, filterResp3.imag,
                        filterResp4.real, filterResp4.imag])

    ## Perform quantization and compute LPQ codewords
    inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
    LPQdesc=((freqResp>0)*(2**inds)).sum(2)

#     ## Switch format to uint8 if LPQ code np.image is required as output
#     if mode=='im':
#         LPQdesc=np.uint8(LPQdesc)


#         pyplot.figure(figsize=(20,10))
#         pyplot.subplot(121)
#         pyplot.imshow(img, cmap='gray')
#         pyplot.title("Original image")

#         pyplot.subplot(122)
#         pyplot.imshow(LPQdesc, cmap='gray')
#         pyplot.title("lpq")
#         pyplot.show()


    ## Histogram if needed
    if mode=='nh' or mode=='h':
        LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

    ## Normalize histogram if needed
    if mode=='nh':
        LPQdesc=LPQdesc/LPQdesc.sum()

    return LPQdesc

In [11]:
def Train_NN(X,Y):
    print("Start Training ")
    N = len(X[0])
    X = np.array(X)
    Y = np.array(Y)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=4, stratify=Y)
    clf = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(512,128),random_state=1,solver='lbfgs',max_iter=10000)
    clf = clf.fit(x_train, y_train)
    print("Finished Fitting")
    y_pred_test = clf.predict(x_test)
    accuracy_test = np.mean(y_test==y_pred_test) * 100
    
    y_pred_train = clf.predict(x_train)
    accuracy_train = np.mean(y_train==y_pred_train) * 100
    return accuracy_train,accuracy_test,clf

In [12]:
base_dir='ACdata_base/'

fonts = os.listdir(base_dir)
X,Y=[],[]
for font in fonts:
    h,w = 0,0
    data = os.listdir(base_dir+font)
    print("curFont",font)
    for img in data:
        img_dir = base_dir+font+'/'+img

        features = lpq(img_dir)
        features = features.flatten()
        X.append(features)
        Y.append(int(font))

curFont 8
curFont 5
curFont 6
curFont 7
curFont 9
curFont 3
curFont 2
curFont 1
curFont 4


In [13]:
accuracyTrain,accuracyTest,ModelNN = Train_NN(X,Y)
print(accuracyTrain,accuracyTest)

Start Training 
Finished Fitting
100.0 42.18009478672986
